# Video of Volcano Camera Images During Certain Hours of Each Day

## Purpose:
To create a video from a series of webcam images. Images are selected from the 'database' on the basis of their date, and the time of the day they were taken. For example, all images taken between 0200 and 0300 UTC each day from January 7 to February 6.

## Technical Details:
Cropping an image is done by default. Currently, the only way to not crop an image, is to crop to the 'full size'.

The image crop is set by variables x1, x2, y1, y2. x1,y1 represents the top left of the image (the image coordinate system starts at the top left), and x2,y2 the bottom right. These need to be specified for each view, and checked/adjusted if a camera's view changes.

The cropped images can be contrast enhanced using an algorithm called CLAHE (Contrast Limited Adaptive Histogram Equalization). The variable 'enhance' can be set to True/False depending on whether enhancement is required.

The date-time of each image is written in the top-left of the image.

Directories for images and video need to be specified.

The image 'database' is the GeoNet public ftp site. On an Ubuntu operating system, the ftp site mounts as a directory, and can therefore be searched for appropriate images. The GeoNet public ftp site therefore needs to be mounted for this notebook to run.

## Author:
Steven Sherburn

## Date:
March 2018

In [ ]:
import cv2
import matplotlib.pyplot as plt
import os
import datetime
from datetime import timedelta 
import pytz
import pandas as pd

%matplotlib inline

In [ ]:
def imfdt(imfile):
    #gets a datetime for an image file named like 20180101001002WIWR.jpg
    year = imfile[:4]
    month = imfile[4:6]
    day = imfile[6:8]
    hour = imfile[8:10]
    minute = imfile[10:12]
    second = imfile[12:14]
    dt = pytz.utc.localize(datetime.datetime(int(year), int(month), int(day), int(hour), int(minute), int(second)))
    return (dt)

In [ ]:
def clahe(img):
    # CLAHE (Contrast Limited Adaptive Histogram Equalization)
    #https://stackoverflow.com/questions/19363293/whats-the-fastest-way-to-increase-color-image-contrast-with-opencv-in-python-c
    global img2
    clahe = cv2.createCLAHE(clipLimit=3., tileGridSize=(8,8))

    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)  # convert from BGR to LAB color space
    l, a, b = cv2.split(lab)  # split on 3 different channels

    l2 = clahe.apply(l)  # apply CLAHE to the L-channel

    lab = cv2.merge((l2,a,b))  # merge channels
    img2 = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)  # convert from LAB to BGR

**Set image date range and camera**

In [ ]:
#UTC date range for images to view
date1 = '2018-01-01'
date2 = '2018-04-30'
camera = 'WINR'

**Text for image label**

In [ ]:
font                   = cv2.FONT_HERSHEY_SIMPLEX
bottomLeftCornerOfText = (10,30)
fontScale              = 0.5
fontColor              = (255,255,255)
lineType               = 1

**Set hours for image selection**

In [ ]:
hour1 = 0
hour2 = 1
strhour1 = str(hour1).zfill(2)
strhour2 = str(hour2).zfill(2)

**Enhance the images?**

In [ ]:
enhance = False

**Image crop**

In [ ]:
#WINR focus on 2012 dome, 2018-05-30 onwards
# y1 = 650
# y2 = 900
# x1 = 1550
# x2 = 1950

#WINR focus on 2012 dome, up to 2018-05-29
y1 = 715
y2 = 965
x1 = 1560
x2 = 1960

**Relevant directories**

In [ ]:
#as mounted, this is image directory
camdir = '/run/user/1000/gvfs/ftp:host=ftp.geonet.org.nz/volcano_cams'

#directory to write crops
imgdir = '/home/sherburn/geonet/volcam_images'
crpdir = os.path.join(imgdir, '2012dome_crop')
viddir = os.path.join(imgdir, '2012dome_video')

**Loop through date folders**

In [ ]:
for date in pd.date_range(date1, date2):
    year = date.year
    time_start = pytz.utc.localize(date + timedelta(hours=hour1))
    time_end = pytz.utc.localize(date + timedelta(hours=hour2))
    datestr = date.strftime('%Y%m%d')
    print (datestr)
    dydir = os.path.join(camdir, str(year), datestr, camera)
    
    for file in sorted(os.listdir(dydir)):
        if file.endswith('.jpg'):
            imfile = os.path.join(dydir, file)
            imtime = imfdt(file)
            if (imtime < time_end and imtime > time_start):
                img = cv2.imread(imfile)
                crop = img[y1:y2, x1:x2]
                if enhance:
                    clahe(crop)
                else:
                    img2 = crop
                cv2.putText(img2,imtime.strftime('%Y-%m-%d %H:%M'), bottomLeftCornerOfText, font, fontScale, fontColor, lineType)
                crfile = os.path.join(crpdir, file)
                cv2.imwrite(crfile, img2)
height,width,layers = crop.shape

**Make video of cropped images**

In [ ]:
vidname = camera+'_'+date1+'_'+date2+'_'+strhour1+'-'+strhour2+'.avi'
vidfile = os.path.join(viddir, vidname)

fps = 6
fourcc = cv2.VideoWriter_fourcc(*'XVID')
video = cv2.VideoWriter(vidfile,fourcc,fps,(width,height))
for file in sorted(os.listdir(crpdir)):
    if file.endswith('.jpg'):
        imfile = os.path.join(crpdir, file)
        img = cv2.imread(imfile)
        video.write(img)
cv2.destroyAllWindows
video.release()